## Mortality Data

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def pre_process_mortality_feature_list(data):
        # dropping columns which are not required
    data.drop(['ph_dipstick','ethnicity','admission_type','mixed_venous_o2_sat','first_careunit','age_bucket','icu_los','edadmittime','eddischargetime'],axis = 1, inplace = True)

    # Cleaning the data by adding limits to remove unrealistic data due to human/techincal errors

    condition_1 = data['non_invasive_blood_pressure_diastolic'] > 200
    data.loc[condition_1, 'non_invasive_blood_pressure_diastolic'] = data.loc[condition_1, 'non_invasive_blood_pressure_mean']
    data = data[(data['heart_rate_mean'] >= 0) & (data['heart_rate_mean'] <= 500)]
    data = data[(data['respiratory_rate'] >= 0) & (data['respiratory_rate'] <= 100)]
    data = data[(data['temperature_fahrenheit'] >= 0) & (data['temperature_fahrenheit'] <= 250)]
    data = data[(data['admission_weight_lbs'] >= 0) & (data['admission_weight_lbs'] <= 1000)]
    data = data[(data['o2_saturation_pulseoxymetry2'] >= 0) & (data['o2_saturation_pulseoxymetry2'] <= 100)]
    data = data[(data['height_cm'] >= 0) & (data['height_cm'] <= 300)]
    data.drop(['non_invasive_blood_pressure_mean'],axis = 1, inplace = True)

    df_encoded = pd.get_dummies(data, columns=['admission_type_combined', 'first_careunit_combined','ethnicity_combined','gender'])

    mean_value = df_encoded['non_invasive_blood_pressure_diastolic'].mean()

    # Replace NaNs in the column with the calculated mean value
    df_encoded['non_invasive_blood_pressure_diastolic'].fillna(mean_value, inplace=True)
    df_encoded['gcs_eye_opening'].fillna(0, inplace=True)
    df_encoded['gcs_verbal_response'].fillna(0, inplace=True)
    df_encoded['gcs_motor_response'].fillna(0, inplace=True)

    df_encoded.drop(columns=['subject_id','hadm_id','hospital_expire_flag'],inplace = True)
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted

def pre_process_mortality(data):    
    # dropping columns which are not required
    data.drop(['ph_dipstick','hospital_expire_flag','admission_type','mixed_venous_o2_sat','first_careunit','age_bucket','icu_los','edadmittime','eddischargetime'],axis = 1, inplace = True)

    # Cleaning the data by adding limits to remove unrealistic data due to human/techincal errors

    condition_1 = data['non_invasive_blood_pressure_diastolic'] > 200
    data.loc[condition_1, 'non_invasive_blood_pressure_diastolic'] = data.loc[condition_1, 'non_invasive_blood_pressure_mean']
    data = data[(data['heart_rate_mean'] >= 0) & (data['heart_rate_mean'] <= 500)]
    data = data[(data['respiratory_rate'] >= 0) & (data['respiratory_rate'] <= 100)]
    data = data[(data['temperature_fahrenheit'] >= 0) & (data['temperature_fahrenheit'] <= 250)]
    data = data[(data['admission_weight_lbs'] >= 0) & (data['admission_weight_lbs'] <= 1000)]
    data = data[(data['o2_saturation_pulseoxymetry2'] >= 0) & (data['o2_saturation_pulseoxymetry2'] <= 100)]
    data = data[(data['height_cm'] >= 0) & (data['height_cm'] <= 300)]
    data.drop(['non_invasive_blood_pressure_mean'],axis = 1, inplace = True)
    
    df_encoded = pd.get_dummies(data, columns=['admission_type_combined', 'first_careunit_combined','ethnicity_combined','gender'])
    
    mean_value = df_encoded['non_invasive_blood_pressure_diastolic'].mean()

    # Replace NaNs in the column with the calculated mean value
    df_encoded['non_invasive_blood_pressure_diastolic'].fillna(mean_value, inplace=True)
    df_encoded['gcs_eye_opening'].fillna(0, inplace=True)
    df_encoded['gcs_verbal_response'].fillna(0, inplace=True)
    df_encoded['gcs_motor_response'].fillna(0, inplace=True)
    
    #df_encoded.drop(columns=['subject_id','hadm_id'],inplace = True)
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted

def model_input_mortality(data_1,data_2):
    
    data = data_1
    features_data = data_2
    
    missing_columns = set(features_data.columns)-set(data.columns)

    if missing_columns:
        # Add missing columns and fill them with zeros
        for column in missing_columns:
            data[column] = 0  # Fill with zeros
            
    data.drop(columns=['subject_id','hadm_id','ethnicity','edadmit_time'],inplace = True)
    data=data.sort_index(axis=1)    
    
    return data

In [3]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''select * from factihealth.mimic.mortality_data;''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    features_data = pd.DataFrame(rows, columns=column_names)

    print('Dataframe shape:', features_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (73181, 28)


In [4]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''select * from 
(
select 
distinct b.*,
        TO_DATE(
        '2024-' || EXTRACT(MONTH FROM edadmittime)::VARCHAR || '-' || EXTRACT(DAY FROM edadmittime)::VARCHAR,
        'YYYY-MM-DD'
    ) AS edadmit_time
from 
    factihealth.mimic.ecg_model_dataset a
inner join 
    factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
where ecgtime between edadmittime and eddischargetime
order by ecgtime,a.subject_id
)
where edadmit_time between '2024-02-01' AND '2024-03-03';''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (851, 29)


In [5]:
features_data = pre_process_mortality_feature_list(features_data)
mortal_data = pre_process_mortality(master_data)

In [6]:
df_merge = mortal_data.copy()
df_merge['index'] = range(0, len(df_merge))
df_merge = df_merge[['index','subject_id','hadm_id','age','height_cm','ethnicity','edadmit_time']]

In [7]:
final_data = model_input_mortality(mortal_data,features_data)
final_data['height_cm'] = pd.to_numeric(final_data['height_cm'], errors='coerce')  # 'coerce' will convert non-numeric values to NaN

In [8]:
import pickle
with open('icu_patients_mortality_prediction.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

In [9]:
mortality_model = loaded_model
prediction_probabilities = mortality_model.predict_proba(final_data)
probabilities_df = pd.DataFrame(prediction_probabilities).reset_index()
mortality_pred = pd.merge(df_merge,probabilities_df,on='index',how ='inner')
mortality_pred.drop(columns = 'index',inplace = True)
mortality_pred.rename(columns={0:'Alive',1:'Dead'},inplace = True)
        #
# Exclude the index column from the DataFrame
mortality_pred = mortality_pred.copy()
mortality_pred.index = [''] * len(mortality_pred)
        #
mortality_pred['height_cm'] = mortality_pred['height_cm'].astype(float)
mortality_pred['Alive'] = mortality_pred['Alive'].astype(float)
mortality_pred['Dead']  = mortality_pred['Dead'].astype(float)
        #
mortality_pred['height_cm'] = mortality_pred['height_cm'].apply(lambda x: f'{x:.1f}')
mortality_pred['Alive'] = mortality_pred['Alive'].apply(lambda x: f'{x:.1%}')
#mortality_pred['Dead'] = mortality_pred['Dead'].apply(lambda x: f'{x:.1%}')
        #
# Display the formatted DataFrame using st.table()
mortality_pred.rename(columns={'height_cm':'Height','subject_id':'Patient ID',
                                                   'hadm_id':'Admission ID',
                                                    'ethnicity':'Ethnicity','edadmit_time':'ED: Admission Date',
                                                    'Alive':'Probabilty of being Alive'
                                                    #'Dead':'Probabilty of being Dead'
                                                    },inplace=True)
        #
mortality_pred = mortality_pred[['ED: Admission Date', 'Admission ID', 'Patient ID', 'Probabilty of being Alive']]

In [10]:
mortality_pred=mortality_pred.sort_values(by='ED: Admission Date', ascending=True)
mortality_pred

,ED: Admission Date,Admission ID,Patient ID,Probabilty of being Alive
,2024-02-01,24827043,15172022,1.3%
,2024-02-01,23980451,18888450,100.0%
,2024-02-01,24605479,11847575,100.0%
,2024-02-01,20035700,15878712,99.6%
,2024-02-01,21609961,14619207,98.8%
...,...,...,...,...
,2024-03-03,26137394,13766350,99.8%
,2024-03-03,21349306,11651470,97.0%
,2024-03-03,23369047,12339424,100.0%
,2024-03-03,29177293,18676703,100.0%


## ECG Data

In [11]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''select * from 
(
select 
distinct a. subject_id,b.hadm_id,
a.gender,
a.anchor_age,
TO_DATE('2024-' || EXTRACT(MONTH FROM b.edadmittime)::VARCHAR || '-' || EXTRACT(DAY FROM b.edadmittime)::VARCHAR,'YYYY-MM-DD') AS edadmit_time,
TO_DATE('2024-' || EXTRACT(MONTH FROM a.ecgtime)::VARCHAR || '-' || EXTRACT(DAY FROM a.ecgtime)::VARCHAR,'YYYY-MM-DD') AS ecg_time,
a.bandwidth,
a.filtering,
a.rr_interval,
a.p_onset,
a.p_end,
a.qrs_onset,
a.qrs_end,
a.t_end,
a.p_axis,
a.qrs_axis,
a.t_axis
from 
factihealth.mimic.ecg_model_dataset a
inner join 
factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
where ecgtime between b.edadmittime and b.eddischargetime
order by ecgtime,a.subject_id
);''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    features_data = pd.DataFrame(rows, columns=column_names)

    print('Dataframe shape:', features_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (13152, 17)


In [12]:
features_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13152 entries, 0 to 13151
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   subject_id    13152 non-null  int64 
 1   hadm_id       13152 non-null  int64 
 2   gender        13152 non-null  object
 3   anchor_age    13152 non-null  int64 
 4   edadmit_time  13152 non-null  object
 5   ecg_time      13152 non-null  object
 6   bandwidth     13152 non-null  object
 7   filtering     13152 non-null  object
 8   rr_interval   13152 non-null  int64 
 9   p_onset       13152 non-null  int64 
 10  p_end         13152 non-null  int64 
 11  qrs_onset     13152 non-null  int64 
 12  qrs_end       13152 non-null  int64 
 13  t_end         13152 non-null  int64 
 14  p_axis        13152 non-null  int64 
 15  qrs_axis      13152 non-null  int64 
 16  t_axis        13152 non-null  int64 
dtypes: int64(12), object(5)
memory usage: 1.7+ MB


In [13]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''select * from 
(
select 
distinct a. subject_id,b.hadm_id,
a.gender,
a.anchor_age,
TO_DATE('2024-' || EXTRACT(MONTH FROM b.edadmittime)::VARCHAR || '-' || EXTRACT(DAY FROM b.edadmittime)::VARCHAR,'YYYY-MM-DD') AS edadmit_time,
TO_DATE('2024-' || EXTRACT(MONTH FROM a.ecgtime)::VARCHAR || '-' || EXTRACT(DAY FROM a.ecgtime)::VARCHAR,'YYYY-MM-DD') AS ecg_time,
a.bandwidth,
a.filtering,
a.rr_interval,
a.p_onset,
a.p_end,
a.qrs_onset,
a.qrs_end,
a.t_end,
a.p_axis,
a.qrs_axis,
a.t_axis
from 
factihealth.mimic.ecg_model_dataset a
inner join 
factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
where ecgtime between b.edadmittime and b.eddischargetime
order by ecgtime,a.subject_id
)
where ecg_time between '2024-02-01' AND '2024-03-03';''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (1146, 17)


In [14]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146 entries, 0 to 1145
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   subject_id    1146 non-null   int64 
 1   hadm_id       1146 non-null   int64 
 2   gender        1146 non-null   object
 3   anchor_age    1146 non-null   int64 
 4   edadmit_time  1146 non-null   object
 5   ecg_time      1146 non-null   object
 6   bandwidth     1146 non-null   object
 7   filtering     1146 non-null   object
 8   rr_interval   1146 non-null   int64 
 9   p_onset       1146 non-null   int64 
 10  p_end         1146 non-null   int64 
 11  qrs_onset     1146 non-null   int64 
 12  qrs_end       1146 non-null   int64 
 13  t_end         1146 non-null   int64 
 14  p_axis        1146 non-null   int64 
 15  qrs_axis      1146 non-null   int64 
 16  t_axis        1146 non-null   int64 
dtypes: int64(12), object(5)
memory usage: 152.3+ KB


In [15]:
def pre_process_ecg_feature_list(data):
    data=data.drop(columns=['hadm_id'])
    df_encoded = pd.get_dummies(data, columns=['bandwidth','filtering','gender'])
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted

def pre_process_ecg(data):
    data=data.drop(columns=['hadm_id'])
    df_encoded = pd.get_dummies(data, columns=['bandwidth','filtering','gender'])
    df_sorted = df_encoded.sort_index(axis=1)
    return df_sorted
    
def model_input_ecg(data_1,data_2):
    
    data = data_1
    features_data = data_2
    
    missing_columns = set(features_data.columns)-set(data.columns)

    if missing_columns:
        # Add missing columns and fill them with zeros
        for column in missing_columns:
            data[column] = 0  # Fill with zeros
    data.drop(columns=['subject_id','ecg_time','edadmit_time'],inplace = True)
    data=data.sort_index(axis=1)
    
    return data

In [16]:
features_data1= features_data.copy()
features_data = pre_process_ecg_feature_list(features_data)
ecg_data = pre_process_ecg(master_data)

In [17]:
df_merge = features_data1
df_merge['index'] = range(0, len(df_merge))
ecg_df_merge = df_merge[['index','subject_id','hadm_id','edadmit_time','ecg_time']]
final_data = model_input_ecg(ecg_data,features_data)

In [18]:
import pickle
with open('C:/Users/krishika.R/Desktop/factihealth-streamlit-demo/factihealth-streamlit-demo/models/icu_patients_ecg_prediction.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

In [19]:
ecg_model = loaded_model
ecg_output=pd.DataFrame(ecg_model.predict(final_data)).reset_index()

ecg_output.rename(columns={0:'ECG Results'},inplace = True)
class_mapping = {0:'Normal ECG', 1: 'Abnormal ECG' }
ecg_output['ECG Results'].replace(class_mapping, inplace=True)
        #
ecg_pred = pd.merge(ecg_df_merge,ecg_output,on='index',how ='inner')
ecg_pred.drop(columns = 'index',inplace = True)
ecg_pred.rename(columns={0:'Alive',1:'Dead'},inplace = True)
        #
# Display the formatted DataFrame using st.table()
ecg_pred.rename(columns={'subject_id':'Patient ID','hadm_id':'Admission ID',
                                            'edadmit_time':'ED: Admission Date',
                                            'ecg_time':'ECG Performed On'
                                            }
                                            ,inplace=True)
        

In [26]:
ecg_pred =ecg_pred.sort_values(by='ED: Admission Date', ascending=True)
ecg_pred

,Patient ID,Admission ID,ED: Admission Date,ECG Performed On,ECG Results
108,14548167,22770469,2024-01-01,2024-01-01,Normal ECG
902,11124675,25900933,2024-01-01,2024-01-26,Abnormal ECG
903,11124675,25900933,2024-01-01,2024-01-28,Abnormal ECG
789,17585185,20858819,2024-01-01,2024-01-01,Abnormal ECG
212,18740413,27577192,2024-01-02,2024-01-02,Abnormal ECG
...,...,...,...,...,...
905,17889152,25398136,2024-12-30,2024-01-31,Abnormal ECG
107,12859052,29332500,2024-12-30,2024-12-30,Abnormal ECG
111,10071869,20895291,2024-12-31,2024-01-10,Normal ECG
207,11676805,25163580,2024-12-31,2024-12-31,Abnormal ECG


In [21]:
#mortality_pred.to_csv('mortality_pred.csv')
#ecg_pred.to_csv('ecg_pred.csv')

In [32]:
ed_merged_data = mortality_pred.merge(ecg_pred,on=['Admission ID'], how = 'inner')

In [33]:
ed_merged_data.drop(columns=['Patient ID_y','ED: Admission Date_y'],inplace=True)
ed_merged_data.rename(columns={'ED: Admission Date_x': 'ED: Admission Date',
                             'Patient ID_x': 'Patient ID'},inplace = True)
ed_merged_data.sort_values(by='ED: Admission Date',inplace=True)

mortal_ecg_dataset = ed_merged_data[['ED: Admission Date','Admission ID','Patient ID','Probabilty of being Alive','ECG Performed On','ECG Results']]
mortal_ecg_dataset

,ED: Admission Date,Admission ID,Patient ID,Probabilty of being Alive,ECG Performed On,ECG Results
0,2024-02-01,20500267,19025096,100.0%,2024-02-01,Normal ECG
1,2024-02-02,27600572,11247575,92.8%,2024-02-02,Abnormal ECG
2,2024-02-02,27600572,11247575,92.8%,2024-02-04,Abnormal ECG
3,2024-02-02,27600572,11247575,92.8%,2024-02-06,Abnormal ECG
4,2024-02-02,20792774,15700698,3.0%,2024-02-21,Abnormal ECG
5,2024-02-04,22379130,17636206,97.5%,2024-02-05,Normal ECG
6,2024-02-04,22379130,17636206,97.5%,2024-02-05,Normal ECG
7,2024-02-04,22379130,17636206,97.5%,2024-02-07,Normal ECG
8,2024-02-04,22379130,17636206,97.5%,2024-02-06,Abnormal ECG
9,2024-02-04,29246638,12153677,99.6%,2024-02-04,Abnormal ECG


In [34]:
mortal_ecg_dataset.to_csv('mortal_ecg_dataset.csv')